<a href="https://colab.research.google.com/github/unmtransinfo/chembl/blob/main/chembl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install required packages

In [ ]:
!pip install chembl-downloader